## MCP Autogen

In [1]:
import asyncio
from common import LLM,MCPActions
from autogen_ext.tools.mcp import SseMcpToolAdapter, SseServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [2]:
server_params = SseServerParams(
        url="http://localhost:8000/mcp",
        timeout=30,  # Connection timeout in seconds
    )

## add individual MCP tool manually

In [4]:
adapter = await SseMcpToolAdapter.from_server_params(server_params, "get_stat_agewise")
adapter1 = await SseMcpToolAdapter.from_server_params(server_params, "get_countries")

agent = AssistantAgent(
        name="extractor",
        model_client=LLM().llm_client,
        tools=[adapter,adapter1],
        system_message="You are a helpful data extractor agent.",
    )


## Discover tools present in MCP server

In [ ]:

from autogen_ext.tools.mcp import create_mcp_server_session
async with create_mcp_server_session(server_params) as session:
    await session.initialize()
    tools_response = await session.list_tools()
    for t in tools_response.tools:
        adpt = await SseMcpToolAdapter.from_server_params(server_params, t.name)
        print(t.name)
    


## load all MCP tools from an MCP server

In [6]:
from autogen_ext.tools.mcp import create_mcp_server_session
adpts = []
async with create_mcp_server_session(server_params) as session:
    await session.initialize()
    tools_response = await session.list_tools()
    for t in tools_response.tools:
        adpt = await SseMcpToolAdapter.from_server_params(server_params, t.name)
        adpts.append(adpt)

agent = AssistantAgent(
        name="extractor",
        model_client=LLM().llm_client,
        tools=adpts,
        system_message="You are a helpful data extractor agent.",
    )
    

## load specific MCP tool from env 

In [ ]:
adpts = []
actions = MCPActions().get_mcp_actions()

for action in actions:
    adpt = await SseMcpToolAdapter.from_server_params(server_params, action)
    adpts.append(adpt)

agent = AssistantAgent(
        name="extractor",
        model_client=LLM().llm_client,
        tools=adpts,
        system_message="""
        You are a extractor agent who will call the MCP tool only.
        #steps
        # Call the MCP Tool with Parameter given by user. Identify the parameter and pass this to the api to find the output.
        Example 1:
        If user asks for how many person from Country: Canda fly through Airport : Delhi
        Call : /data/airport?country=Canada&airport=Delhi

        Example 2:
        If user asks how many person from Country: Canada cam betwwen age 10 and 30
        call : /agewise?country=Canada&age_from=10&age_to=30

        #output format
        Example 1:
        If the response code of the Stool is 200 then provide a response

        Example 2:
        If The response code of MCP tool is 204 then out message : 'No records found'
        """,
    )
    

In [ ]:
response = await agent.on_messages(
    [TextMessage(content="How many person visited from Canada via Delhi airport",source="user")],
    cancellation_token=CancellationToken()
)

print(response.chat_message.content)

1503 travellers came from Czech between the age of 45 to 54.
